In [1]:
import sys
sys.path.append('..')
from src.read_databases import read_database4, read_database3, get_database4, transform_database4, get_database3, transform_database3
import pandas as pd
import numpy as np
import h5py
import yaml
from scipy.io import loadmat
import os
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow.parquet import ParquetFile

In [2]:
h = 9
w = 14

for subject in [1,2]:
    # if 'dado4' in locals() and (subject-1)%4==0:
    #     del(dado4)  
    for segment in ['forearm', 'biceps', 'triceps']:
        for task in ['s','e','f','p']:
            for intensity in [10,30, 50]:
                print(f'{subject=},{segment=},{task=},{intensity=}')
                dado_new = transform_database4(h, w, subject, segment, task, intensity, trial=1, session=None, pathcwd='../')
                if 'dado4' in locals():
                    dado4 = pd.concat([dado4, dado_new], join='outer')
                else:
                    dado4 = dado_new.copy()
    
    # if (subject)%4==0 or subject == 12:
    #     if os.path.exists('../data/processed/dado.parquet'):
    #         table = pa.Table.from_pandas(dado4, schema=pqwriter.schema)
    #         pqwriter.write_table(table)
    #     else:
    #         table = pa.Table.from_pandas(dado4)
    #         pqwriter = pq.ParquetWriter('../data/processed/dado.parquet', table.schema, compression='snappy')   
    #         pqwriter.write_table(table)

subject=1,segment='forearm',task='s',intensity=10
subject=1,segment='forearm',task='s',intensity=30
subject=1,segment='forearm',task='s',intensity=50
subject=1,segment='forearm',task='e',intensity=10
subject=1,segment='forearm',task='e',intensity=30
subject=1,segment='forearm',task='e',intensity=50
subject=1,segment='forearm',task='f',intensity=10
subject=1,segment='forearm',task='f',intensity=30
subject=1,segment='forearm',task='f',intensity=50
subject=1,segment='forearm',task='p',intensity=10
subject=1,segment='forearm',task='p',intensity=30
subject=1,segment='forearm',task='p',intensity=50
subject=1,segment='biceps',task='s',intensity=10
subject=1,segment='biceps',task='s',intensity=30
subject=1,segment='biceps',task='s',intensity=50
subject=1,segment='biceps',task='e',intensity=10
subject=1,segment='biceps',task='e',intensity=30
subject=1,segment='biceps',task='e',intensity=50
subject=1,segment='biceps',task='f',intensity=10
subject=1,segment='biceps',task='f',intensity=30
subject=

In [3]:
h = 9
w = 14

for subject in range(1,26):
    if 'dado4' in locals() and (subject-1)%2==0 and subject !=1 or subject==2:
       
        del(dado4)  
    for trial in range(1, 7):
        for task in range(1,14):
            for session in range(1,4):
                print(f'{subject=},{trial=},{task=},{session=}')
                dado_new = transform_database3(h, w, subj=subject, type='forearm', task=task, intensity=None, trial=trial, session=session, pathcwd='../')                
                if 'dado4' in locals():
                    dado4 = pd.concat([dado4, dado_new], join='outer')
                else:
                    dado4 = dado_new.copy()
    # dado3 = pd.concat([dado4_head, dado3], join='outer')
    # dado_new = dado3.query('database_id==3')
    # dado4_head = pd.concat([dado4_head, dado3.iloc[0:10,:]], join='outer')
    # dado4_head = dado4_head.query('database_id==4')
    # dado3['trial'] = dado3['trial'].astype('int')
    if (subject)%2==0 or subject == 25 or subject == 1:
        if os.path.exists('../data/processed/dado.parquet'):
            table = pa.Table.from_pandas(dado4, schema=pqwriter.schema)
            pqwriter.write_table(table)
        else:
            table = pa.Table.from_pandas(dado4)
            pqwriter = pq.ParquetWriter('../data/processed/dado.parquet', table.schema, compression='gzip')   
            pq
            writer.write_table(table)

# dado3.to_parquet('../data/processed/dado3.parquet', engine='fastparquet', index=False) 
del(dado4) 

subject=1,trial=1,task=1,session=1
subject=1,trial=1,task=1,session=2
subject=1,trial=1,task=1,session=3
subject=1,trial=1,task=2,session=1
subject=1,trial=1,task=2,session=2
subject=1,trial=1,task=2,session=3
subject=1,trial=1,task=3,session=1
subject=1,trial=1,task=3,session=2
subject=1,trial=1,task=3,session=3
subject=1,trial=1,task=4,session=1
subject=1,trial=1,task=4,session=2
subject=1,trial=1,task=4,session=3
subject=1,trial=1,task=5,session=1
subject=1,trial=1,task=5,session=2
subject=1,trial=1,task=5,session=3
subject=1,trial=1,task=6,session=1
subject=1,trial=1,task=6,session=2
subject=1,trial=1,task=6,session=3
subject=1,trial=1,task=7,session=1
subject=1,trial=1,task=7,session=2
subject=1,trial=1,task=7,session=3
subject=1,trial=1,task=8,session=1
subject=1,trial=1,task=8,session=2
subject=1,trial=1,task=8,session=3
subject=1,trial=1,task=9,session=1
subject=1,trial=1,task=9,session=2
subject=1,trial=1,task=9,session=3
subject=1,trial=1,task=10,session=1
subject=1,trial=1,t

In [4]:
h = 9
w = 14

for subject in [3,4,5,6,7,8,9,10,11,12]:
    # if 'dado4' in locals() and (subject-1)%4==0:
    #     del(dado4)  
    for segment in ['forearm', 'biceps', 'triceps']:
        for task in ['s','e','f','p']:
            for intensity in [10,30, 50]:
                print(f'{subject=},{segment=},{task=},{intensity=}')
                dado_new = transform_database4(h, w, subject, segment, task, intensity, trial=1, session=None, pathcwd='../')
                if 'dado4' in locals():
                    dado4 = pd.concat([dado4, dado_new], join='outer')
                else:
                    dado4 = dado_new.copy()

subject=3,segment='forearm',task='s',intensity=10
subject=3,segment='forearm',task='s',intensity=30
subject=3,segment='forearm',task='s',intensity=50
subject=3,segment='forearm',task='e',intensity=10
subject=3,segment='forearm',task='e',intensity=30
subject=3,segment='forearm',task='e',intensity=50
subject=3,segment='forearm',task='f',intensity=10
subject=3,segment='forearm',task='f',intensity=30
subject=3,segment='forearm',task='f',intensity=50
subject=3,segment='forearm',task='p',intensity=10
subject=3,segment='forearm',task='p',intensity=30
subject=3,segment='forearm',task='p',intensity=50
subject=3,segment='biceps',task='s',intensity=10
subject=3,segment='biceps',task='s',intensity=30
subject=3,segment='biceps',task='s',intensity=50
subject=3,segment='biceps',task='e',intensity=10
subject=3,segment='biceps',task='e',intensity=30
subject=3,segment='biceps',task='e',intensity=50
subject=3,segment='biceps',task='f',intensity=10
subject=3,segment='biceps',task='f',intensity=30
subject=

In [5]:
table = pa.Table.from_pandas(dado4, schema=pqwriter.schema)
pqwriter.write_table(table)

KeyError: "name 'length (cm)' present in the specified schema is not found in the columns or index"

In [ ]:
pqwriter.close()

In [6]:
h = 9
w = 14

for subject in [1]:
    for trial in range(1, 7):
        for task in range(1,14):
            for session in range(1,4):
                print(f'{subject=},{trial=},{task=},{session=}')
                dado_new = transform_database3(h, w, subj=subject, type='forearm', task=task, intensity=None, trial=trial, session=session, pathcwd='../')                
                if 'dado_novo' in locals():
                    dado_novo = pd.concat([dado_novo, dado_new], join='outer')
                else:
                    dado_novo = dado_new.copy()

subject=1,trial=1,task=1,session=1
subject=1,trial=1,task=1,session=2
subject=1,trial=1,task=1,session=3
subject=1,trial=1,task=2,session=1
subject=1,trial=1,task=2,session=2
subject=1,trial=1,task=2,session=3
subject=1,trial=1,task=3,session=1
subject=1,trial=1,task=3,session=2
subject=1,trial=1,task=3,session=3
subject=1,trial=1,task=4,session=1
subject=1,trial=1,task=4,session=2
subject=1,trial=1,task=4,session=3
subject=1,trial=1,task=5,session=1
subject=1,trial=1,task=5,session=2
subject=1,trial=1,task=5,session=3
subject=1,trial=1,task=6,session=1
subject=1,trial=1,task=6,session=2
subject=1,trial=1,task=6,session=3
subject=1,trial=1,task=7,session=1
subject=1,trial=1,task=7,session=2
subject=1,trial=1,task=7,session=3
subject=1,trial=1,task=8,session=1
subject=1,trial=1,task=8,session=2
subject=1,trial=1,task=8,session=3
subject=1,trial=1,task=9,session=1
subject=1,trial=1,task=9,session=2
subject=1,trial=1,task=9,session=3
subject=1,trial=1,task=10,session=1
subject=1,trial=1,t

In [7]:
dado_novo = pd.concat([dado_novo, dado4], join='outer')
dado_novo = dado_novo.query('database_id==4')

In [8]:
del(dado4)

In [10]:
dado_novo['database_id']

0        4
1        4
2        4
3        4
4        4
        ..
20475    4
20476    4
20477    4
20478    4
20479    4
Name: database_id, Length: 7335936, dtype: int8

In [11]:
table = pa.Table.from_pandas(dado_novo, schema=pqwriter.schema)
pqwriter.write_table(table)

In [ ]:
pqwriter.close()